# Live Assignment 6
## DS 6001: Practice and Application of Data Science
### Drew Haynes (rbc6wr)

#### Robert's Super Secret Docker Formula 

docker run -e POSTGRES_PASSWORD=mysecretpassword -d postgres

https://t.co/hA5qJcfGoG


### Building a Database of NBA Games, Teams, Players, and Box Scores

A box score is a tabular representation of everything that happened during a professional basketball game in the NBA. For example, here is the box score for a recent game that took place between the Cleveland Cavaliers and the Milwaukee Bucks:

<img src="https://raw.githubusercontent.com/jkropko/DS-6001/master/localimages/boxscore.png"
     alt="Figure 12.1"
     width="600" />
     
For this lab we will be working with data that collects and appends all box scores from all NBA games together. The data were compiled by Stewart Gibson of [Advanced Sports Analytics](https://www.advancedsportsanalytics.com/about) and are available here: https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/. To access the most recent data, right-click on "Raw Data Download" and copy the download link. Save this URL in your Jupyter notebook and pass it to `pd.read_csv()` to load the data.

Unfortunately, the data download link changes every day. There is an advanced web scraping problem here: we can't use `BeautifulSoup` because the download URL does not appear in the source HTML code for the website. We can use selenium, an alternative and more aggressive web-scraping package that connects with web browsers on your machine to extract exactly what appears on the screen. However that's beyond our scope today as we are focusing on databases. For now, be prepared to update the URL every few hours if you are going to develop this project.

Powerful website: https://dbdocs.io/

The raw data has 81 columns. Stewart Gibson does not provide a data dictionary, but Dr. Kropko has written one here:

|Column No.|Column name|Data type|Varies By|Calculated from other columns?|Description|
|--- |--- |--- |--- |--- |--- |
|0|game_id|object|Game|No|Unique ID for one NBA game|
|1|game_date|object|Game|No|Date in YYYY-MM-DD format|
|2|OT|int64|Game|No|Boolean: did the game go into overtime?|
|3|H_A|object|Game and Team|No|Is the team playing at home or away in this game?|
|4|Team_Abbrev|object|Team|No|Team|
|5|Team_Score|int64|Team|No|Team's score in this game|
|6|Team_pace|float64|Game and Team|No|Pace Calculation Formula=[240/(Team Minutes)]*(Possession_team + Possession_opponent)/2|
|7|Team_efg_pct|float64|Game and Team|Yes|Effective Field Goal Percentage Formula=[(All Field Goals Made) + 0.5*(3P Field Goals Made)]/(All Field Goal Attempts)|
|8|Team_tov_pct|float64|Game and Team|Yes|Turnover Ratio Formula=(Turnovers)100)/ [(Field Goal Attempts)+(Free Throw Attempts0.44)+(Assists)+(Turnovers)]|
|9|Team_orb_pct|float64|Game and Team|Yes|Offensive Rebounding Percentage Formula=(Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)]|
|10|Team_ft_rate|float64|Game and Team|Yes|Free Throw Rate Formula= (Total Free Throws Attempted) / (Total Field Goal Attempted)]|
|11|Team_off_rtg|float64|Game and Team|No|Offensive Efficiency Formula=100*(Points Scored / Possessions)|
|12|Inactives|object|Game and Team|Yes|List of inactive players|
|13|Opponent_Abbrev|object|Game and Team|No|Opponent team|
|14|Opponent_Score|int64|Game and Team|No|Opponent's score|
|15|Opponent_pace|float64|Game and Team|Yes|Pace Calculation Formula=[240/(Team Minutes)]*(Possession_team + Possession_opponent)/2|
|16|Opponent_efg_pct|float64|Game and Team|Yes|Effective Field Goal Percentage Formula=[(All Field Goals Made) + 0.5*(3P Field Goals Made)]/(All Field Goal Attempts)|
|17|Opponent_tov_pct|float64|Game and Team|Yes|Turnover Ratio Formula=(Turnovers)100)/ [(Field Goal Attempts)+(Free Throw Attempts0.44)+(Assists)+(Turnovers)]|
|18|Opponent_orb_pct|float64|Game and Team|Yes|Offensive Rebounding Percentage Formula=(Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)]|
|19|Opponent_ft_rate|float64|Game and Team|Yes|Free Throw Rate Formula= (Total Free Throws Attempted) / (Total Field Goal Attempted)]|
|20|Opponent_off_rtg|float64|Game and Team|No|Offensive Efficiency Formula=100*(Points Scored / Possessions)|
|21|player|object|Player|No|Name of player|
|22|player_id|object|Player|No|Unique ID for the player|
|23|starter|int64|Game and Player|No|Binary: did this player start the game?|
|24|mp|object|Game and Player|No|Minutes played in MM:SS format|
|25|fg|int64|Game and Player|No|Number of field goals (shots other than free throws) made|
|26|fga|int64|Game and Player|No|Number of field goals (shots other than free throws) attempted|
|27|fg_pct|float64|Game and Player|Yes|Field goal percentage formula = (fg / fga)|
|28|fg3|int64|Game and Player|No|Three-point field goals made|
|29|fg3a|int64|Game and Player|No|Three-point field goals attempted|
|30|fg3_pct|float64|Game and Player|Yes|Three-point field goal percentage formula = (fg3 / fg3a)|
|31|ft|int64|Game and Player|No|Free throws made|
|32|fta|int64|Game and Player|No|Free throws attempted|
|33|ft_pct|float64|Game and Player|Yes|Free throw percentage formula = (ft / fta)|
|34|orb|int64|Game and Player|No|Offensive rebounds|
|35|drb|int64|Game and Player|No|Defensive rebounds|
|36|trb|int64|Game and Player|Yes|Total rebounds|
|37|ast|int64|Game and Player|No|Assists|
|38|stl|int64|Game and Player|No|Steals|
|39|blk|int64|Game and Player|No|Blocks|
|40|tov|int64|Game and Player|No|Turnovers|
|41|pf|int64|Game and Player|No|Personal fouls|
|42|pts|int64|Game and Player|No|Points|
|43|plus_minus|int64|Game and Player|No|Total team score minus total opponent team score for the minutes in which this player was in the game|
|44|did_not_play|int64|Game and Player|No|1 if the player did not play, 0 if the player played at all|
|45|is_inactive|int64|Game and Player|No|1 if the player was inactive, 0 if the player was available to play|
|46|ts_pct|float64|Game and Player|Yes|True Shooting Percentage Formula=0.5(Total Points)/[(Total Field Goal Attempts) + 0.44(Total Free Throw Attempts)]|
|47|efg_pct|float64|Game and Player|Yes|Effective Field Goal Percentage Formula=[(All Field Goals Made) + 0.5*(3P Field Goals Made)]/(All Field Goal Attempts)|
|48|fg3a_per_fga_pct|float64|Game and Player|Yes|Formula = (fg3a / fga)|
|49|fta_per_fga_pct|float64|Game and Player|Yes|Formula = (fta / fga)|
|50|orb_pct|float64|Game and Player|Yes|Formula = 100 * (player's orb / team orb)|
|51|drb_pct|float64|Game and Player|Yes|Formula = 100 * (player's drb / team drb)|
|52|trb_pct|float64|Game and Player|Yes|Formula = 100 * (player's trb / team trb)|
|53|ast_pct|float64|Game and Player|Yes|Formula = 100 * (player's ast / team ast)|
|54|stl_pct|float64|Game and Player|Yes|Formula = 100 * (player's stl / team stl)|
|55|blk_pct|float64|Game and Player|Yes|Formula = 100 * (player's blk / team blk)|
|56|tov_pct|float64|Game and Player|Yes|Formula = 100 * (player's tov / team tov)|
|57|usg_pct|float64|Game and Player|Yes|Usage rate, aka NBA usage percentage, is an estimate of the  percentage of team plays used by a player while he was on the floor.  Usage rate can be calculated with how a possession ends: “field-goal or  free-throw attempt, or a turnover. Usage Rate Calculation Formula = 100((Player’s Field Goal Attempts)+0.44(Player’s Free Throw Attempts)+(Player’s Turnovers))(Team’s Total Minutes)/((Team’s Total Field Goal Attempts)+0.44(Team’s Total Free Throw Attempts)+Team’s Total Turnovers))5(Player’s Minutes)|
|58|off_rtg|int64|Game and Player|No|Number of team points per 48 minutes while this player was on the floor.|
|59|def_rtg|int64|Game and Player|No|Number of opponent team's points per 48 minutes while this player was on the floor.|
|60|bpm|float64|Game and Player|No|Box plus-minus takes box score stats from an individual and tries to estimate player performance relative to the NBA average. Since BPM is a per-100-possession stat where 0.0 is league average, +5 means the player is 5 points better than an average player over 100 possessions (which is about All-NBA level), -2 is replacement level, and -5 is really bad.|
|61|season|int64|Game|Yes|Year in which the season ended. For the 2021-2022 season this value is 2022|
|62|minutes|float64|Game and Player|No|Minutes the player was in the game|
|63|double_double|int64|Game and Player|Yes|Binary: did the player record double digits in two of points, rebounds, assists, steals, and blocks?|
|64|triple_double|int64|Game and Player|Yes|Binary: did the player record double digits in three of points, rebounds, assists, steals, and blocks?|
|65|DKP|float64|Game and Player|Yes|Fantasy points scored by this player on DraftKings.com|
|66|FDP|float64|Game and Player|Yes|Fantasy points scored by this player on FanDuel.com|
|67|SDP|float64|Game and Player|Yes|Fantasy points scored by this player on another sports betting website (I'm not sure what SD is though)|
|68|DKP_per_minute|object|Game and Player|Yes|DKP / minutes|
|69|FDP_per_minute|object|Game and Player|Yes|FDP / minutes|
|70|SDP_per_minute|object|Game and Player|Yes|SDP / minutes|
|71|pf_per_minute|float64|Game and Player|Yes|pf / minutes|
|72|ts|float64|N/A|N/A|(Not sure what this column is. We'll delete it)|
|73|last_60_minutes_per_game_starting|float64|N/A|N/A|(Not sure what this column is. We'll delete it)|
|74|last_60_minutes_per_game_bench|float64|N/A|N/A|(Not sure what this column is. We'll delete it)|
|75|PG%|float64|Game and Player|No|% of player's minutes playing the point guard position|
|76|SG%|float64|Game and Player|No|% of player's minutes playing the point guard position|
|77|SF%|float64|Game and Player|No|% of player's minutes playing the point guard position|
|78|PF%|float64|Game and Player|No|% of player's minutes playing the power forward position|
|79|C%|float64|Game and Player|Yes|% of player's minutes playing the center position|
|80|active_position_minutes|float64|N/A|N/A|(Not sure what this column is. We'll delete it)|

In [1]:
import numpy as np
import pandas as pd
import requests

Note: the URL is changing every few hours. First go to https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/ then right click on "Raw Data Download" and copy and paste the new URL below.

In [3]:
url = 'https://stewart-gibson.shinyapps.io/NBA_All_Data_Downloader/_w_f2f86230/session/08fc1e49e704bfdb1e74dac849984af2/download/downloadData?w=f2f86230'

In [9]:
nba = pd.read_csv(url, dtype=object)

In [10]:
nba

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202280BRK,2022-02-28,0,A,TOR,133,101.1,0.606,9.9,34,...,0.0293111871030777,15.76,33.6909722222222,0,0,0,29,62,9,NaN
1,202202280BRK,2022-02-28,0,A,TOR,133,101.1,0.606,9.9,34,...,0.029747149231532,11.88,8.29634043100525,4.46666666666667,99,1,0,0,0,NaN
2,202202280BRK,2022-02-28,0,A,TOR,133,101.1,0.606,9.9,34,...,0.1,15.32,39.4083333333333,21.2170523079126,0,0,0,55,45,NaN
3,202202280BRK,2022-02-28,0,A,TOR,133,101.1,0.606,9.9,34,...,0.0338028169014085,13.08,34.5106060606061,18.5801649557787,16,83,1,0,0,NaN
4,202202280BRK,2022-02-28,0,A,TOR,133,101.1,0.606,9.9,34,...,0.314685314685315,1,17.3190476190476,13.4015151515152,0,0,0,0,100,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112856,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7,18.9,...,0.107913669,13.08,33.11066667,19.23256217,0,2,77,21,0,57.20778588
112857,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7,18.9,...,0.036079375,6,25.47083333,20.22857143,5,45,43,7,0,58.20239132
112858,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7,18.9,...,0.150943396,4,24.08333333,13.22878788,0,0,0,9,91,49.63063965
112859,202003070GSW,2020-03-07,0,H,GSW,118,90.9,0.606,7,18.9,...,0.094339623,12.64,34.78333333,27.69166667,0,44,48,8,0,58.92351503


In [16]:
pd.options.display.max_rows = None # Normally don't want to do unless you have very few rows
nba.head(1).T


,0
game_id,202202280BRK
game_date,2022-02-28
OT,0
H_A,A
Team_Abbrev,TOR
Team_Score,133
Team_pace,101.1
Team_efg_pct,0.606
Team_tov_pct,9.9
Team_orb_pct,34


### First Normal Form (1NF)

For a table to be in the First Normal Form, it should follow the following 4 rules:

1. It should only have single(atomic) valued attributes/columns.
2. Values stored in a column should be of the same domain
3. All the columns in a table should have unique names.
4. And the order in which data is stored, does not matter.

https://www.studytonight.com/dbms/first-normal-form.php

Primary Key (PK) = game_id + player_id

However, non-atomic values are stored in `Inactives` so it is currently not in 1NF. However, we can reproduce this field with `is_inactive`.

In [23]:
if 'Inactives' in nba.columns :
    nba = nba.drop('Inactives', axis = 1)

We do not have repeated columns, so we're okay for Rule #3. If we did, we could either a) create arbitrary columns, i.e. if we had multiple authors, we could instead have "Author 1, Author 2" etc. But that will create a lot of missing data. Alternatively, we could create a new table that links books to authors with a many to many cardinality. 

Attribute means column, why do they have to say attribute?

### Second Normal Form (2NF)

For a table to be in the Second Normal Form,

1. It should be in the First Normal form.
2. And, it should not have Partial Dependency.

https://www.studytonight.com/dbms/second-normal-form.php

`OT` is a functionally dependent on `game_id`, but not `player_id` so it is in violation. We could change it to the index, if we wanted to.

### Third Normal Form (3NF)

A table is said to be in the Third Normal Form when,

1. It is in the Second Normal form.
2. And, it doesn't have Transitive Dependency.

https://www.studytonight.com/dbms/third-normal-form.php

Treatment: Take out the dependents, only leave those that are depended on.

Five Tables

1. Information overall about the game: OT, date, location, etc.

2. Info about how the team did overall in the game.

3. Info about how the player did personally in the game.

4. Info about the team's total season stats so far.

5. Info about the player's total season so far.

### Boyce and Codd Normal Form (BCNF)

Boyce and Codd Normal Form is a higher version of the Third Normal form. This form deals with certain type of anomaly that is not handled by 3NF. A 3NF table which does not have multiple overlapping candidate keys is said to be in BCNF. For a table to be in BCNF, following conditions must be satisfied:

1. It must be in 3rd Normal Form
2. and, for each functional dependency ( X → Y ), X should be a super Key.

https://www.studytonight.com/dbms/boyce-codd-normal-form.php

### Fourth Normal Form (4NF)

A table is said to be in the Fourth Normal Form when,

1. It is in the Boyce-Codd Normal Form.
2. And, it doesn't have Multi-Valued Dependency.

https://www.studytonight.com/dbms/fourth-normal-form.php